In [3]:
from pathlib import Path
import pandas as pd
import pickle
import gzip

In [4]:
sif_path = Path('/data/MLDSST/nielsond/cogmood/analysis/SUPREME/Analysis/Flanker/flanker.sif')
swarm_dir = Path('/data/MLDSST/nielsond/cogmood/swarm/')
swarm_cmd_dir = swarm_dir / 'cmds'
swarm_log_dir = swarm_dir / 'log'
clean_csv_dir = Path('../data/task/to_model_first100').resolve()
dq = pd.read_csv(clean_csv_dir / 'data_quality.csv')
res_dir = Path('../data/task/flkr_test_first100').resolve()
res_dir.mkdir(exist_ok=True)

In [5]:
good_flkr_csvs = [clean_csv_dir / f'flkr-{sid}.csv' for sid in dq.loc[dq.good10_flkr, 'sub_id'].values]

In [6]:
cmds = []
for sub_file in good_flkr_csvs:
    break

In [10]:
cmds = []
for sub_file in good_flkr_csvs:
    subject = sub_file.stem.replace('flanker_', '').replace('flkr-', '').replace('.csv','')
    cmd = f"""rsync -avch /data/MLDSST/nielsond/cogmood/analysis/cogmood_analysis/packages/Supreme/Analysis/Flanker /lscratch/$SLURM_JOBID/; \
    rsync -avch {sub_file} /lscratch/$SLURM_JOBID/Flanker/; \
    cd /lscratch/$SLURM_JOBID/Flanker; \
    mkdir -p /lscratch/$SLURM_JOBID/cache/pycuda; \
    ls -ltah /lscratch/$SLURM_JOBID/Flanker/flanker.sif; \
    export MPLBACKEND=pdf; \
    apptainer run --nv -B /lscratch/$SLURM_JOBID/Flanker:/Flanker \
        -B /lscratch/$SLURM_JOBID/cache:/data/nielsond/.cache \
        /lscratch/$SLURM_JOBID/Flanker/flanker.sif {sub_file.parts[-1]} ; \
        ls ; \
    rsync -avch flkr_res_{subject}.tgz {res_dir}/; \
    rsync -avch flkr_map_{subject}.csv {res_dir}/; 
    """
    cmds.append(cmd)

In [11]:
cmd_file = swarm_cmd_dir/'flkr_cmds_f100'
cmd_file.write_text('\n'.join(cmds[2:]))

97509

In [12]:

! swarm --time=12:00:00 -g 30 -t 20 -m apptainer --gres=gpu:1,lscratch:5 --partition=gpu --job-name=flkr_f100 --logdir={swarm_log_dir} --qos=gpunimh2025.1 --sbatch="--constraint=gpuk80\|gpuv100" {cmd_file}

11834519


In [13]:
def load_results(filename):
    """
    Load in a simulation that was saved to a pickle.gz.
    """
    gf = gzip.open(filename, 'rb')
    res = pickle.loads(gf.read(), encoding='latin1')
    gf.close()
    return res


In [17]:
cmds = []
for sub_file in good_flkr_csvs:
    result_good=False
    subject = sub_file.stem.replace('flanker_', '').replace('flkr-', '').replace('.csv','')
    outfile = res_dir / f'flkr_res_{subject}.tgz'
    if outfile.exists():
        nsims = load_results(outfile)['particles'].shape[0]
        if nsims == 2001:
            result_good = True
    if not result_good:
        cmd = f"""rsync -avch /data/MLDSST/nielsond/cogmood/analysis/cogmood_analysis/packages/Supreme/Analysis/Flanker /lscratch/$SLURM_JOBID/; \
    rsync -avch {sub_file} /lscratch/$SLURM_JOBID/Flanker/; \
    cd /lscratch/$SLURM_JOBID/Flanker; \
    mkdir -p /lscratch/$SLURM_JOBID/cache/pycuda; \
    ls -ltah /lscratch/$SLURM_JOBID/Flanker/flanker.sif; \
    export MPLBACKEND=pdf; \
    apptainer run --nv -B /lscratch/$SLURM_JOBID/Flanker:/Flanker \
        -B /lscratch/$SLURM_JOBID/cache:/data/nielsond/.cache \
        /lscratch/$SLURM_JOBID/Flanker/flanker.sif {sub_file.parts[-1]} ; \
        ls ; \
    rsync -avch flkr_res_{subject}.tgz {res_dir}/; \
    rsync -avch flkr_map_{subject}.csv {res_dir}/; 
    """
        cmds.append(cmd)

In [18]:
len(cmds)

2

In [19]:
cmd_file = swarm_cmd_dir/'flkr6_fails_f100'
cmd_file.write_text('\n'.join(cmds))

1989

In [20]:
! swarm --time=12:00:00 -g 30 -t 20 -m apptainer --gres=gpu:1,lscratch:5 --partition=gpu --job-name=flkr_f100 --logdir={swarm_log_dir} --qos=gpunimh2025.1 --sbatch="--constraint=gpuk80\|gpuv100" {cmd_file}

11949594


Just some commands that might be needed for debuging:

```
apptainer shell --nv -B /lscratch/$SLURM_JOBID/Flanker:/Flanker -B /home/nielsond:/home/nielsond -B /lscratch/$SLURM_JOBID/cache:/data/nielsond/.cache ./flanker.sif 

sinteractive --gres=gpu:k80:1,lscratch:20 --mem=40g -c20 --time=24:00:00
```